In [2]:
import os
import sys
import yaml
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf
from model.fp.nnfp import get_fingerprinter
from model.dataset import Dataset
from model.fp.melspec.melspectrogram import get_melspec_layer
from model.fp.specaug_chain.specaug_chain import get_specaug_chain_layer
import numpy as np
from model.fp.NTxent_loss_single_gpu import NTxentLoss

In [7]:
def load_config(config_fname):
    config_filepath = './config/' + config_fname + '.yaml'
    if os.path.exists(config_filepath):
        print(f'cli: Configuration from {config_filepath}')
    else:
        sys.exit(f'cli: ERROR! Configuration file {config_filepath} is missing!!')

    with open(config_filepath, 'r') as f:
        cfg = yaml.safe_load(f)
    return cfg

In [8]:
config:str = "default"  
cfg = load_config(config)

checkpoint_root_dir = '/mnt/dev/rodrigoalmeida/NeuralNetworks/neural-audio-fp/logs/checkpoint/'
checkpoint_name:str = 'CHECKPOINT_BSZ_120'
checkpoint_index:int = 100

cli: Configuration from ./config/default.yaml


In [4]:
def load_checkpoint(checkpoint_root_dir, checkpoint_name, checkpoint_index,
                    m_fp):
    """ Load a trained fingerprinter """
    # Create checkpoint
    checkpoint = tf.train.Checkpoint(model=m_fp)
    checkpoint_dir = checkpoint_root_dir + f'/{checkpoint_name}/'
    c_manager = tf.train.CheckpointManager(checkpoint, checkpoint_dir,
                                           max_to_keep=None)

    # Load
    if checkpoint_index == None:
        tf.print("\x1b[1;32mArgument 'checkpoint_index' was not specified.\x1b[0m")
        tf.print('\x1b[1;32mSearching for the latest checkpoint...\x1b[0m')
        latest_checkpoint = c_manager.latest_checkpoint
        if latest_checkpoint:
            checkpoint_index = int(latest_checkpoint.split(sep='ckpt-')[-1])
            status = checkpoint.restore(latest_checkpoint)
            status.expect_partial()
            tf.print(f'---Restored from {c_manager.latest_checkpoint}---')
        else:
            raise FileNotFoundError(f'Cannot find checkpoint in {checkpoint_dir}')
    else:
        checkpoint_fpath = checkpoint_dir + 'ckpt-' + str(checkpoint_index)
        status = checkpoint.restore(checkpoint_fpath) # Let TF to handle error cases.
        status.expect_partial()
        tf.print(f'---Restored from {checkpoint_fpath}---')
    return checkpoint_index

In [5]:
"""checkpoint = tf.train.Checkpoint(model=meuModelo)
c_manager = tf.train.CheckpointManager(checkpoint, checkpoint_root_dir+checkpoint_name, max_to_keep=None)

latest_checkpoint = c_manager.latest_checkpoint
latest_checkpoint
checkpoint_index = int(latest_checkpoint.split(sep='ckpt-')[-1])
checkpoint_fpath =  checkpoint_root_dir+checkpoint_name + 'ckpt-' + str(checkpoint_index)
status = checkpoint.restore(checkpoint_fpath)"""

"checkpoint = tf.train.Checkpoint(model=meuModelo)\nc_manager = tf.train.CheckpointManager(checkpoint, checkpoint_root_dir+checkpoint_name, max_to_keep=None)\n\nlatest_checkpoint = c_manager.latest_checkpoint\nlatest_checkpoint\ncheckpoint_index = int(latest_checkpoint.split(sep='ckpt-')[-1])\ncheckpoint_fpath =  checkpoint_root_dir+checkpoint_name + 'ckpt-' + str(checkpoint_index)\nstatus = checkpoint.restore(checkpoint_fpath)"

In [9]:
meuModelo = get_fingerprinter(cfg, trainable=False)

#checkpoint_index = load_checkpoint(checkpoint_root_dir, checkpoint_name, 100, meuModelo)

In [37]:
#um = tf.ones(1,256,32,1)
# voltar a carregar o modelo com '1s'

x = tf.constant(np.random.randn(1,256,32), dtype=tf.float32) # BxFxTx1

#input_shape=(256,32,1)
#x = tf.zeros((1,)+ input_shape, dtype=tf.float32)

In [38]:
emb_1s = meuModelo(x)

In [29]:
emb_1s

<tf.Tensor: shape=(1, 128), dtype=float32, numpy=
array([[-8.43031146e-03, -4.22595888e-02, -3.46743576e-02,
         6.59990609e-02,  1.03641469e-02, -1.25604765e-02,
        -2.31130674e-01,  3.93131785e-02, -7.74441585e-02,
        -2.66612824e-02,  2.42525697e-01, -3.90986241e-02,
        -5.49379624e-02,  8.08970481e-02,  9.04842187e-03,
         7.99441636e-02, -1.28806368e-01, -3.56026855e-03,
        -1.71384305e-01, -1.47024821e-03, -2.99457065e-03,
         3.31072621e-02,  5.13421260e-02,  1.02011345e-01,
         7.05138594e-03,  1.90437570e-01,  2.91897301e-02,
        -9.80698410e-03, -1.37220379e-02,  7.39087313e-02,
        -9.21153352e-02, -4.95435260e-02, -1.40590295e-01,
        -3.73935699e-02, -5.80401812e-03, -1.18680738e-01,
        -2.02048682e-02,  3.16333771e-01,  3.47597711e-02,
        -1.29932612e-01,  1.57815382e-01, -2.37651393e-02,
         5.57486601e-02, -9.65843499e-02,  6.71994165e-02,
         6.05744310e-02,  1.01996958e-02,  4.89653349e-02,
      

In [39]:
meuModelo.summary()

Model: "finger_printer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ConvLayers (Sequential)     (None, 1024)              16897920  
                                                                 
 div_enc_layer (DivEncLayer  multiple                  41088     
 )                                                               
                                                                 
Total params: 16939008 (64.62 MB)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 16939008 (64.62 MB)
_________________________________________________________________


In [40]:
pathModel = '/home/rodrigo/Documents/neural-audio-fp/ModeloGuardadoFolder/modelo_savedFormat'

In [22]:
meuModelo.compile()

save_weights

In [23]:
meuModelo.save_weights(pathModel) #meuModelo.load_weights(pathModel)

In [25]:
newModel = get_fingerprinter(cfg, trainable=False)

In [27]:
newModel.load_weights(pathModel)

In [ ]:
newModel.summary()

checkpoint.save

In [22]:
checkpoint = tf.train.Checkpoint(model=meuModelo)
checkpoint.save(pathModel)

'/home/rodrigo/Documents/neural-audio-fp/ModeloGuardadoFolder/modelo_checkpoint/-1'

savedFormat

In [41]:
tf.saved_model.save(meuModelo, pathModel)

TypeError: in user code:

    File "/home/rodrigo/anaconda3/envs/tf/lib/python3.11/site-packages/keras/src/saving/legacy/saving_utils.py", line 147, in _wrapped_model  *
        outputs = model(*args, **kwargs)
    File "/home/rodrigo/anaconda3/envs/tf/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_filec9iexoq6.py", line 11, in tf__call
        x = ag__.converted_call(ag__.ld(self).div_enc, (ag__.ld(x),), None, fscope)
    File "/tmp/__autograph_generated_filev8k10k_g.py", line 10, in tf__call
        x = ag__.converted_call(ag__.ld(tf).reshape, (ag__.ld(x),), dict(shape=[ag__.ld(x).shape[0], ag__.ld(self).q, -1]), fscope)

    TypeError: Exception encountered when calling layer 'finger_printer' (type FingerPrinter).
    
    in user code:
    
        File "/home/rodrigo/Documents/neural-audio-fp/model/fp/nnfp.py", line 230, in call  *
           

save

In [ ]:
meuModelo.save('/home/rodrigo/Documents/neural-audio-fp/ModeloGuardadoFolder/modelo_save')

In [ ]:
meuModelo.front_conv.summary()

Passar Subclassing para Functional

In [12]:
meuModelo.div_enc.weights

[<tf.Variable 'dense/kernel:0' shape=(8, 32) dtype=float32, numpy=
 array([[ 0.38673759,  0.37435377,  0.0140616 , -0.12025002,  0.32228613,
          0.34427303, -0.0672071 ,  0.28880268, -0.03173363, -0.35257718,
         -0.2560358 , -0.0518412 , -0.10963506,  0.30398262, -0.15809824,
         -0.1212301 , -0.36001378, -0.29963657,  0.29633534,  0.29596728,
         -0.2888356 ,  0.02413854, -0.10819605, -0.36177647, -0.21442224,
         -0.1700497 ,  0.35918736,  0.21913707, -0.19785883,  0.12469512,
         -0.17991908, -0.1878885 ],
        [ 0.38154256,  0.17308784, -0.18945254, -0.3361036 ,  0.29316533,
         -0.33664507, -0.13726386,  0.3040185 ,  0.33096492, -0.35818058,
         -0.34801   ,  0.06684428, -0.30221337,  0.23957872, -0.07153973,
          0.2762407 ,  0.34392327,  0.37320757, -0.02010629, -0.3769309 ,
         -0.18670592, -0.22080001,  0.23432094, -0.10219818,  0.24793285,
         -0.15163071,  0.07653573, -0.18386815,  0.30462766,  0.33292466,
         

In [13]:
a=meuModelo.get_weights

In [14]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, Input

#@tf.function
def convlayer(hidden_ch=128,
              strides=[(1,1),(1,1)],
              norm='layer_norm2d'):
    conv2d_1x3 = tf.keras.layers.Conv2D(hidden_ch,
                                        kernel_size=(1, 3),
                                        strides=strides[0],
                                        padding='SAME',
                                        dilation_rate=(1, 1),
                                        kernel_initializer='glorot_uniform',
                                        bias_initializer='zeros')
    conv2d_3x1 = tf.keras.layers.Conv2D(hidden_ch,
                                        kernel_size=(3, 1),
                                        strides=strides[1],
                                        padding='SAME',
                                        dilation_rate=(1, 1),
                                        kernel_initializer='glorot_uniform',
                                        bias_initializer='zeros')
    if norm == 'layer_norm1d':
        BN_1x3 = tf.keras.layers.LayerNormalization(axis=-1)
        BN_3x1 = tf.keras.layers.LayerNormalization(axis=-1)
    elif norm == 'layer_norm2d':
        BN_1x3 = tf.keras.layers.LayerNormalization(axis=(1, 2, 3))
        BN_3x1 = tf.keras.layers.LayerNormalization(axis=(1, 2, 3))
    else:
        BN_1x3 = tf.keras.layers.BatchNormalization(axis=-1)
        BN_3x1 = tf.keras.layers.BatchNormalization(axis=-1)
        
    forward = tf.keras.Sequential([conv2d_1x3,
                                   tf.keras.layers.ELU(),
                                   BN_1x3,
                                   conv2d_3x1,
                                   tf.keras.layers.ELU(),
                                   BN_3x1
                                   ])
    
    return forward


#@tf.function
def create_sequential_front_conv(input_shape=(256,32,1),
                                 emb_sz=128,
                                 front_hidden_ch=[128, 128, 256, 256, 512, 512, 1024, 1024],
                                 front_strides=[[(1,2), (2,1)], [(1,2), (2,1)],
                                                [(1,2), (2,1)], [(1,2), (2,1)],
                                                [(1,1), (2,1)], [(1,2), (2,1)],
                                                [(1,1), (2,1)], [(1,2), (2,1)]],
                                 norm='layer_norm2d'):
    front_conv = tf.keras.Sequential(name='ConvLayers')
    if ((front_hidden_ch[-1] % emb_sz) != 0):
        front_hidden_ch[-1] = ((front_hidden_ch[-1]//emb_sz) + 1) * emb_sz

    for i in range(len(front_strides)):
        front_conv.add(convlayer(hidden_ch=front_hidden_ch[i], strides=front_strides[i], norm=norm))
    front_conv.add(tf.keras.layers.Flatten())

    return front_conv


#@tf.function
def auxiliar(input1):
    conv_layer = create_sequential_front_conv(input_shape=(256,32,1),
                                               emb_sz=128,
                                               front_hidden_ch=[128, 128, 256, 256, 512, 512, 1024, 1024],
                                               front_strides=[[(1,2), (2,1)], [(1,2), (2,1)],
                                                              [(1,2), (2,1)], [(1,2), (2,1)],
                                                              [(1,1), (2,1)], [(1,2), (2,1)],
                                                              [(1,1), (2,1)], [(1,2), (2,1)]],
                                               norm='layer_norm2d')

    unit_dim = [32, 1]
    q = 128
    arquiteturas_densas = tf.keras.Sequential([tf.keras.layers.Dense(unit_dim[0], activation='elu'),
                                               tf.keras.layers.Dense(unit_dim[1])])

    x = input1
    #x reshape
    x = conv_layer(x)

    y_list = [0] * q
    x_split = tf.split(x, num_or_size_splits=128, axis=1)#x_split = tf.unstack(x, axis=1)

    #for v, k in enumerate(x_split):
    #    y_list[v] = arquiteturas_densas(k)
    y_list = []
    x_split = tf.split(x, num_or_size_splits=q, axis=1)

    for k in x_split:
        y_list.append(arquiteturas_densas(k))

    out = tf.concat(y_list, axis=1)
    output = tf.math.l2_normalize(out, axis=1)
    return output

def get_fingerprinting(input_shape=(256,32,1)):
    input1 = Input(shape=input_shape)
    output = auxiliar(input1)
    fingerprinting_model = Model(inputs=input1, outputs=output)
    return fingerprinting_model

In [15]:
dataset = Dataset(cfg)

In [16]:
total_nsteps = cfg['TRAIN']['MAX_EPOCH'] * len(dataset.get_train_ds())
lr_schedule = tf.keras.experimental.CosineDecay(
    initial_learning_rate=float(cfg['TRAIN']['LR']),
    decay_steps=total_nsteps,
    alpha=1e-06)

# Optimizer
opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)


# Loss objects
if cfg['LOSS']['LOSS_MODE'].upper() == 'NTXENT': # Default
    loss_obj_train = NTxentLoss(
        n_org=cfg['BSZ']['TR_N_ANCHOR'],
        n_rep=cfg['BSZ']['TR_BATCH_SZ'] - cfg['BSZ']['TR_N_ANCHOR'],
        tau=cfg['LOSS']['TAU'])
    loss_obj_val = NTxentLoss(
        n_org=cfg['BSZ']['VAL_N_ANCHOR'],
        n_rep=cfg['BSZ']['VAL_BATCH_SZ'] - cfg['BSZ']['VAL_N_ANCHOR'],
        tau=cfg['LOSS']['TAU'])

In [17]:
def build_fp(cfg):
    """ Build fingerprinter """
    # m_pre: log-power-Mel-spectrogram layer, S.
    m_pre = get_melspec_layer(cfg, trainable=False)

    # m_specaug: spec-augmentation layer.
    m_specaug = get_specaug_chain_layer(cfg, trainable=False)
    assert(m_specaug.bypass==False) # Detachable by setting m_specaug.bypass.

    # m_fp: fingerprinter g(f(.)).
    m_fp = get_fingerprinter(cfg, trainable=False)
    return m_pre, m_specaug, m_fp


@tf.function
def train_step(X, m_pre, m_specaug, m_fp, loss_obj):
    """ Train step """
    n_anchors = len(X[0])
    X = tf.concat(X, axis=0)
    feat = m_specaug(m_pre(X))  # (nA+nP, F, T, 1)
    m_fp.trainable = True
    #functional_model=get_fingerprinting(feat)
    with tf.GradientTape() as t:
        #functional_model=get_fingerprinting(feat)
        emb = m_fp(feat)  # (BSZ, Dim)
        loss, sim_mtx, _ = loss_obj.compute_loss(
            emb[:n_anchors, :], emb[n_anchors:, :]) # {emb_org, emb_rep}
    g = t.gradient(loss, m_fp.trainable_variables)
    return loss, sim_mtx

In [18]:
#@tf.function
def func():
    m_pre, m_specaug, m_fp = build_fp(cfg)
    ep_start = 0
    ep_max = 1
    for ep in range(ep_start, ep_max + 1):
        tf.print(f'EPOCH: {ep}/{ep_max}')

        # Train
        train_ds = dataset.get_train_ds(cfg['DATA_SEL']['REDUCE_ITEMS_P'])
        enq = tf.keras.utils.OrderedEnqueuer(
            train_ds, use_multiprocessing=True, shuffle=train_ds.shuffle)
        enq.start(workers=cfg['DEVICE']['CPU_N_WORKERS'],
                    max_queue_size=cfg['DEVICE']['CPU_MAX_QUEUE'])
        i = 0
        while i < len(enq.sequence):
            X = next(enq.get()) # X: Tuple(Xa, Xp)
            avg_loss, sim_mtx = train_step(X, m_pre, m_specaug, m_fp,
                                            loss_obj_train)
            i += 1
        enq.stop()

In [ ]:
func()

In [20]:
input1 = tf.keras.Input(shape=(256, 32, 1))

In [ ]:
functional_model=get_fingerprinting(input1)
functional_model.set_weights(a)

Pesos

In [8]:
dirWeights = '/home/rodrigo/Documents/neural-audio-fp/ModeloGuardadoFolder/'